In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('data/train.csv')
df = df.drop(columns='id')

test_df = pd.read_csv('data/test.csv')
test_df = test_df.drop(columns='id')
display(df.head())
display(test_df.head())

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [3]:
X = df.drop(labels='Premium Amount', axis=1)
Y = df['Premium Amount']

In [4]:
# Hist gb
hist_X = X.copy()
hist_X['Policy Year'] = pd.to_datetime(hist_X['Policy Start Date']).dt.year
hist_X = hist_X.drop('Policy Start Date', axis=1)
hist_X= hist_X.fillna('None').astype('string')


k = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
oof = np.zeros(len(hist_X))

for index, (train_index, test_index) in enumerate(k.split(hist_X)):
    hist_model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.1,
            depth=6,
            eval_metric="RMSE",
            random_seed=42,
            verbose=200,
            task_type='GPU',
            l2_leaf_reg =  0.7,
        )

    hist_model.fit(hist_X.iloc[train_index],
                   Y.iloc[train_index],
                   early_stopping_rounds=300,
                   cat_features = hist_X.columns.values)
    predictions = hist_model.predict(hist_X.iloc[test_index])
    oof[test_index] = predictions

0:	learn: 863.0677044	total: 89.8ms	remaining: 1m 29s
200:	learn: 844.1707151	total: 3.99s	remaining: 15.9s
400:	learn: 842.7862059	total: 7.92s	remaining: 11.8s
600:	learn: 841.7360194	total: 11.9s	remaining: 7.92s
800:	learn: 840.7988481	total: 15.9s	remaining: 3.96s
999:	learn: 839.9238670	total: 19.9s	remaining: 0us
0:	learn: 863.3785419	total: 18.5ms	remaining: 18.5s
200:	learn: 844.4502710	total: 3.96s	remaining: 15.7s
400:	learn: 843.0492971	total: 8.09s	remaining: 12.1s
600:	learn: 841.9306017	total: 12.2s	remaining: 8.09s
800:	learn: 841.0532267	total: 16.4s	remaining: 4.07s
999:	learn: 840.2020735	total: 20.4s	remaining: 0us
0:	learn: 862.5566611	total: 18.3ms	remaining: 18.3s
200:	learn: 843.4974605	total: 3.79s	remaining: 15s
400:	learn: 842.1429731	total: 7.76s	remaining: 11.6s
600:	learn: 841.2123822	total: 11.8s	remaining: 7.83s
800:	learn: 840.2755205	total: 15.8s	remaining: 3.92s
999:	learn: 839.4706650	total: 19.9s	remaining: 0us
0:	learn: 863.0018135	total: 18.3ms	re

In [5]:
nonneg_oof = np.where(oof < 0, 0, oof)
np.sqrt(mean_squared_log_error(nonneg_oof, Y))

1.1413753072867034

In [6]:
new_X = hist_X.copy()
new_X['nonlog'] = nonneg_oof

kf = KFold(n_splits=5, shuffle=True, random_state=42)
second_oof = np.zeros(len(new_X))
models = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(new_X)):
    print(f"Fold {fold + 1}")
    X_train, X_valid = new_X.iloc[train_idx], new_X.iloc[valid_idx]
    y_train, y_valid = np.log1p(Y).iloc[train_idx], np.log1p(Y).iloc[valid_idx]

    model = CatBoostRegressor(
        iterations=3000,
        learning_rate=0.05,
        depth=6,
        eval_metric="RMSE",
        random_seed=42,
        verbose=200,
        task_type='GPU',
        l2_leaf_reg =  0.7,
    )

    model.fit(X_train,
              y_train,
              eval_set=(X_valid, y_valid), 
              early_stopping_rounds=300,
              cat_features=hist_X.columns.values,
             )
    models.append(model)
    oof[valid_idx] = np.maximum(0, model.predict(X_valid))
    fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(oof[valid_idx])))
    print(f"Fold {fold + 1} RMSLE: {fold_rmsle}")

Fold 1
0:	learn: 1.0927908	test: 1.0935319	best: 1.0935319 (0)	total: 44.8ms	remaining: 2m 14s
200:	learn: 1.0483496	test: 1.0471396	best: 1.0471396 (200)	total: 4.79s	remaining: 1m 6s
400:	learn: 1.0430231	test: 1.0403235	best: 1.0403235 (400)	total: 9.58s	remaining: 1m 2s
600:	learn: 1.0405062	test: 1.0374704	best: 1.0374704 (600)	total: 14.1s	remaining: 56.4s
800:	learn: 1.0389231	test: 1.0359713	best: 1.0359703 (798)	total: 18.7s	remaining: 51.4s
1000:	learn: 1.0377042	test: 1.0350783	best: 1.0350779 (999)	total: 23.3s	remaining: 46.5s
1200:	learn: 1.0366056	test: 1.0343245	best: 1.0343245 (1200)	total: 27.9s	remaining: 41.9s
1400:	learn: 1.0356470	test: 1.0337650	best: 1.0337639 (1397)	total: 32.5s	remaining: 37.1s
1600:	learn: 1.0348006	test: 1.0333598	best: 1.0333598 (1600)	total: 37.1s	remaining: 32.4s
1800:	learn: 1.0339903	test: 1.0330044	best: 1.0330032 (1796)	total: 41.7s	remaining: 27.8s
2000:	learn: 1.0331978	test: 1.0327245	best: 1.0327245 (2000)	total: 46.2s	remaining: 

In [13]:
error = np.sqrt(mean_squared_error(oof, np.log1p(Y)))
error

1.029949774566434

# Try without the extra feature

In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
third_oof = np.zeros(len(new_X))
models2 = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(hist_X)):
    print(f"Fold {fold + 1}")
    X_train, X_valid = hist_X.iloc[train_idx], hist_X.iloc[valid_idx]
    y_train, y_valid = np.log1p(Y).iloc[train_idx], np.log1p(Y).iloc[valid_idx]

    model = CatBoostRegressor(
        iterations=3000,
        learning_rate=0.05,
        depth=6,
        eval_metric="RMSE",
        random_seed=42,
        verbose=200,
        task_type='GPU',
        l2_leaf_reg =  0.7,
    )

    model.fit(X_train,
              y_train,
              eval_set=(X_valid, y_valid), 
              early_stopping_rounds=300,
              cat_features=hist_X.columns.values,
             )
    models2.append(model)
    third_oof[valid_idx] = np.maximum(0, model.predict(X_valid))
    fold_rmsle = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(third_oof[valid_idx])))
    print(f"Fold {fold + 1} RMSLE: {fold_rmsle}")

Fold 1
0:	learn: 1.0938368	test: 1.0942692	best: 1.0942692 (0)	total: 17ms	remaining: 51s
200:	learn: 1.0657389	test: 1.0610682	best: 1.0610682 (200)	total: 3.7s	remaining: 51.5s
400:	learn: 1.0641071	test: 1.0599384	best: 1.0599384 (400)	total: 7.6s	remaining: 49.3s
600:	learn: 1.0633074	test: 1.0595971	best: 1.0595971 (600)	total: 11.6s	remaining: 46.4s
800:	learn: 1.0626825	test: 1.0594413	best: 1.0594411 (796)	total: 15.8s	remaining: 43.3s
1000:	learn: 1.0620581	test: 1.0592857	best: 1.0592835 (996)	total: 19.9s	remaining: 39.8s
1200:	learn: 1.0614954	test: 1.0592024	best: 1.0592024 (1200)	total: 24.4s	remaining: 36.5s
1400:	learn: 1.0609025	test: 1.0591408	best: 1.0591408 (1400)	total: 28.6s	remaining: 32.7s
1600:	learn: 1.0603409	test: 1.0590722	best: 1.0590716 (1593)	total: 32.9s	remaining: 28.8s
1800:	learn: 1.0597683	test: 1.0590036	best: 1.0590033 (1799)	total: 37.3s	remaining: 24.8s
2000:	learn: 1.0592181	test: 1.0589589	best: 1.0589581 (1997)	total: 41.6s	remaining: 20.8s
2

# Explore feature importance

In [19]:
for i, model in enumerate(models2):
    # Get feature importance
    importances = model.get_feature_importance()
    feature_names = hist_X.columns

    # Display feature importance
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    print(f"Model {i}")
    print(importance_df)

Model 0
                 Feature  Importance
2          Annual Income   29.057157
7           Health Score   23.607576
12          Credit Score   16.429884
10       Previous Claims    5.108043
0                    Age    3.467235
14     Customer Feedback    3.259897
11           Vehicle Age    2.693383
18           Policy Year    2.563114
13    Insurance Duration    2.234084
4   Number of Dependents    1.844582
3         Marital Status    1.836767
6             Occupation    1.443717
16    Exercise Frequency    1.421732
5        Education Level    1.336237
8               Location    1.093716
9            Policy Type    1.077221
17         Property Type    1.035732
15        Smoking Status    0.266211
1                 Gender    0.223712
Model 1
                 Feature  Importance
2          Annual Income   29.907905
7           Health Score   24.113345
12          Credit Score   17.067500
10       Previous Claims    5.371054
14     Customer Feedback    3.428511
0                    A